# **Simple integration of DP Langevin equation**

Run a basic DP simulation.

<hr>

## Preliminaries

In [ ]:
from essentials import *
dplvn.__version__

In [ ]:
density_dict: dict[float, NDArray] = {}
density_image_dict: dict[int, Any] = {}

<hr>

## Notes

In [ ]:
fetch_image("Snippet_Henkel2008_Table4p1_APTScalingRelations.jpg",)

In [ ]:
fetch_image("Snippet_Henkel2008_Table4p3_DirectedPercolationScaling.jpg",)


<hr>

## Parameters

In [ ]:
random_seed: int = 1
# sim_name_: str = "a1p18855_b1_D0p04_η1_x31_y31_Δx1_Δt0p1"
# sim_name_: str = "a1p18855_b1_D0p04_η1_x62_y62_Δx1_Δt0p1"
sim_name_: str = "a1p18855_b1_D0p04_η1_x125_y125_Δx1_Δt0p1"
# sim_name_: str = "a1p18855_b1_D0p04_η1_x250_y250_Δx1_Δt0p1"
# sim_name_: str = "a1p18855_b1_D0p04_η1_x500_y500_Δx1_Δt0p1"
# sim_name_: str = "a1p18855_b1_D0p04_η1_x1000_y1000_Δx1_Δt0p1"
# sim_name_: str = "a1p18855_b1_D0p04_η1_x2000_y2000_Δx1_Δt0p1"
sim_name = sim_name_ #+ f"_rs{random_seed}"
sim_name

In [ ]:
info: dict = import_info(
    os.path.join(os.path.pardir, "experiments", sim_name,), 
    "Info"
)
analysis: dict = info["Analysis"]
parameters: dict = info["Parameters"]
misc: dict = info["Misc"]
# if sim_name!=set_name(parameters, analysis,):
#     raise NameError

In [ ]:
analysis.update({
    "dp_β": 0.584,
    "dp_ν_pp": 0.734,
    "dp_ν_ll": 1.295,
    "dp_δ": 0.451,
})
make_dataframe(analysis)

In [ ]:
#   linear=1.18850, quadratic=1, diffusion=0.04, noise=1, dx=1, 
# linear=1.18900, quadratic=1, diffusion=0.04, noise=1, dx=1, 
# linear=1.18850, quadratic=1, diffusion=0.01, noise=1, dx=0.5, 
# linear=2.2140, quadratic=1, diffusion=0.1, noise=2*np.sqrt(2),
# linear=1.61, quadratic=1.61, diffusion=0.04, noise=1*1.414,
# linear=1.0, quadratic=2.0, diffusion=0.1, noise=1.1564,
make_dataframe(parameters)

In [ ]:
misc["dplvn_version"] = dplvn.__version__
misc["date_time"] = datetime.now().replace(microsecond=0).isoformat(sep=" ")
make_dataframe(misc)

<hr>

## Simulation

In [ ]:
sim = dplvn.SimDP(**parameters)

In [ ]:
if not sim.initialize(misc["n_round_Δt_summation"]):
    raise Exception("Failed to initialize sim")
t_epochs: NDArray
mean_densities: NDArray
analysis["n_epochs"] = sim.get_n_epochs()
print(f"Number of sim epochs = {analysis["n_epochs"]}")

In [ ]:
def run_sim(n_segments: int=500) -> tuple[NDArray, NDArray]:
    n_epochs: int = sim.get_n_epochs()
    n_segment_epochs: int = (n_epochs-1) // n_segments
    if (n_segment_epochs*n_segments+1)!=n_epochs:
        raise Exception(
            f"Failed to segment sim with {n_epochs} epochs "
            + f"into {n_segments} segment(s)"
        )
    t_epoch_: int
    i_segment_: int
    for i_segment_ in progress(range(0, n_segments+1, 1)):
        if i_segment_>0 and not sim.run(n_segment_epochs):
            raise Exception("Failed to run sim")
        if not sim.postprocess():
            raise Exception("Failed to process sim results")
        # i_epoch = sim.get_i_current_epoch()
        t_epoch_ = sim.get_t_current_epoch()
        density_dict[t_epoch_] = sim.get_density()
    return (sim.get_t_epochs(), sim.get_mean_densities(),)

In [ ]:
tick: float = perf_counter()
%time (t_epochs, mean_densities,) = run_sim(misc["n_segments"])
tock: float = perf_counter()

In [ ]:
misc["computation_time"] = f"{timedelta(seconds=round(tock-tick))}"
print(f"Computation time = {misc["computation_time"]}")

<hr>

## Plot

In [ ]:
graphs = Viz()
images = Viz()

Plot the time-series of grid-averaged density field:

In [ ]:
graphs.plot_mean_density_evolution(
    set_name(
        parameters, analysis, "ρ_t", 
        suffix=f"_rs{parameters["random_seed"]}"+"_loglog",
    ), 
    parameters, analysis, misc, 
    t_epochs, mean_densities, 
    do_rescale=False, y_sf=0.75,
)

In [ ]:
graphs.plot_mean_density_evolution(
    set_name(parameters, analysis, "ρ_t",
             suffix=f"_rs{parameters["random_seed"]}"+"_rescaled",),
    parameters, analysis, misc,
    t_epochs, mean_densities, 
    do_rescale=True,
)

In [ ]:
# graphs.plot_mean_density_evolution(
#     set_name(parameters, analysis, "ρ_t", "_linear"), 
#     parameters, results, misc,
#     t_epochs, mean_densities, 
#     do_loglog=False, do_rescale=False, y_sf=0.75,
# )

Plot image grids of density field time-slices:

In [ ]:
n_digits: int = misc["n_digits"]
name_: str 
density_: NDArray
for i_epoch_, t_epoch_ in progress(enumerate(density_dict.keys())):
    name_ =  f"ρ_t{t_epoch_:0{n_digits}.1f}".replace(".","p")
    # print(i_epoch_, t_epoch_, name_)
    density_ = density_dict[t_epoch_]
    density_image_dict[i_epoch_] = images.plot_density_image(
        name_, 
        parameters, 
        analysis,
        t_epoch_, 
        density_, 
        density_max=3,
        tick_Δρ=1,
        do_extend_if_periodic=False,
        n_digits=n_digits,
    )

<hr>

## Save

In [ ]:
sim_dir_name: str = sim_name

outfo: dict = {
    "Parameters" : parameters,
    "Analysis" : analysis,
    "Misc" : misc
}
serializable_outfo: dict
(serializable_outfo, _,) = export_info(
    create_directories((os.path.pardir, "experiments",), sim_dir_name,), 
    "Outfo", outfo, 
)
pp(serializable_outfo)

In [ ]:
graphs_path: str = export_plots(
    graphs.fdict, 
    create_directories(
        (os.path.pardir, "experiments", sim_dir_name,), 
        "Graphs",
    ),
)

In [ ]:
data_path: str = create_directories(
    (os.path.pardir,"experiments", sim_dir_name,), "Data", do_clean=False,
)
np.savez_compressed(
    os.path.join(data_path, "ρ_t"+f"_rs{parameters["random_seed"]}"), 
    t_epochs=t_epochs,
    mean_densities=mean_densities,
)

In [ ]:
data_npz: NpzFile = np.load(
    os.path.join(data_path, "ρ_t"+f"_rs{parameters["random_seed"]}"+".npz"), 
)
data_npz["t_epochs"][-10:], data_npz["mean_densities"][-10:]

In [ ]:
if "do_export_images" in misc and misc["do_export_images"]:
    images_path: str = export_plots(
        images.fdict, 
        create_directories(
            (os.path.pardir,"experiments", sim_dir_name,), 
            "Images", do_clean=True,
        ),
    )
    print(images_path)

In [ ]:
if "do_make_video" in misc and misc["do_make_video"]:
    videos_path: str = create_directories(
        (os.path.pardir,"experiments", sim_dir_name,), "Videos",
    )

    video_frame_rate: int = misc["video_frame_rate"]
    video_format: str = misc["video_format"]
    video_images_wildcard: str = "ρ_t"+"?"*n_digits+".png"
    input = ffmpeg.input( 
        os.path.join(images_path, video_images_wildcard), 
        pattern_type="glob", 
        framerate=video_frame_rate, 
        pix_fmt="yuv420p",
    )
    output = ffmpeg.output(
        input.video,
        os.path.join(
            videos_path, 
            f"ρ_{sim_dir_name}_rs{parameters["random_seed"]}.{video_format}"
        ),
        vf="crop=floor(iw/2)*2:floor(ih/2)*2",
        vcodec="libx264",
        format=video_format,
    )

In [ ]:
if "do_make_video" in misc and misc["do_make_video"]:
    stderr_output: str = output.overwrite_output().run(capture_stderr=True,)
    # print(stderr_output)